In [1]:
import tensorflow as tf
import numpy as np

2025-11-01 04:27:33.001195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761967653.023832    3415 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761967653.029746    3415 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761967653.045675    3415 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761967653.045700    3415 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761967653.045702    3415 computation_placer.cc:177] computation placer alr

In [14]:
# Parameters
num_features = 28 * 28
num_classes = 10

learning_rate = 0.05
batch_size = 128
training_steps = 1000
display_step = 50

In [15]:
# Dataset
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Data clean up

# To convert the dataset to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1D vector 
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

#Normalize data value from [0, 255], [0, 1]
x_train, x_test = x_train/255, x_test/255

In [16]:
# Shuffling and batching of data
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [17]:
# Assigning weight and bias

W = tf.Variable(tf.random.normal([num_features, num_classes]), name = "weights")
b = tf.Variable(tf.random.normal([num_classes], name="bias"))
print(b.shape)


# The model
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    p_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

(10,)


In [18]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        logits = logistic_regression(x)
        loss = cross_entropy(logits, y)

    gradients = g.gradient(loss, [W, b])

    optimizer.apply_gradients(zip(gradients, [W,b]))


In [19]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)

    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 6.788724, accuracy: 0.250000
step: 100, loss: 5.760701, accuracy: 0.273438
step: 150, loss: 3.774724, accuracy: 0.445312
step: 200, loss: 3.035271, accuracy: 0.507812
step: 250, loss: 3.264692, accuracy: 0.476562
step: 300, loss: 2.756955, accuracy: 0.507812
step: 350, loss: 2.367041, accuracy: 0.625000
step: 400, loss: 2.504009, accuracy: 0.593750
step: 450, loss: 1.848670, accuracy: 0.656250
step: 500, loss: 1.760332, accuracy: 0.656250
step: 550, loss: 1.741433, accuracy: 0.656250
step: 600, loss: 1.520044, accuracy: 0.742188
step: 650, loss: 1.556459, accuracy: 0.664062
step: 700, loss: 1.740805, accuracy: 0.671875
step: 750, loss: 1.811658, accuracy: 0.687500
step: 800, loss: 1.386839, accuracy: 0.750000
step: 850, loss: 2.235796, accuracy: 0.664062
step: 900, loss: 1.032430, accuracy: 0.757812
step: 950, loss: 1.009751, accuracy: 0.804688
step: 1000, loss: 1.013353, accuracy: 0.765625
